# HMM decoder!
See [wiki](https://zh.wikipedia.org/wiki/%E7%BB%B4%E7%89%B9%E6%AF%94%E7%AE%97%E6%B3%95) first, 

Then we are doctors in the hospital! 

In [1]:
states = ('Healthy', 'Fever')
 
observations = ('normal', 'cold', 'dizzy')
 
start_probability = {'Healthy': 0.6, 'Fever': 0.4}
 
transition_probability = {
   'Healthy' : {'Healthy': 0.7, 'Fever': 0.3},
   'Fever' : {'Healthy': 0.4, 'Fever': 0.6},
   }
 
emission_probability = {
   'Healthy' : {'normal': 0.5, 'cold': 0.4, 'dizzy': 0.1},
   'Fever' : {'normal': 0.1, 'cold': 0.3, 'dizzy': 0.6},
   }

transition_probability: $P(s_{t}|s_{t-1})$

emission_probability: $P(o_{t}|s_{t})$

We want $P(s_{0:t}|o_{0:t})$ 

or $P(s_{0:t}, o_{0:t})$ because $P(o_{0:t})$ doesn't matter.

Then HOW?

We can calculate every possibility of sequences, and ***arg* max** it.

eg. 1
\begin{align}
P(s_0, s_0, s_0 , o_0, o_1, o_2) &= P(s_0 | start) \cdot P(o_0 | s_0) \cdot P(s_0 | s_0) \cdot P(o_1 | s_0) \cdot P(s_0 | s_0) \cdot P(o_2 | s_0) \\
&=P(s_0 | start)\cdot \prod_{t=1}^T P(s_t|s_{t-1})P(o_{t-1}|s_{t-1})
\end{align}

...
\begin{align}
\mathop{\arg\max}_s (\pi\cdot \prod_{t=1}^T P(s_t|s_{t-1})P(o_{t-1}|s_{t-1}))
\end{align}

In [2]:
#Healthy, Healthy, Healthy
start_probability['Healthy'] * emission_probability['Healthy'][observations[0]]  \
* transition_probability['Healthy']['Healthy']* emission_probability['Healthy'][observations[1]]  \
* transition_probability['Healthy']['Healthy'] * emission_probability['Healthy'][observations[2]]

0.00588

In [3]:
def brute_force(states, observations, start_probability, transition_probability, emission_probability):
    
    all_binary_s = [bin(i)[2:].zfill(3) for i in range(2**len(observations)-1)]
    
    all_s = []
    for ss in all_binary_s:
        all_s.append([states[int(s)] for s in ss])
        
    print(all_s)
    print('-'*90)
    
    result_score = 0
    result = None

    for s in all_s:

        score = start_probability[s[0]] * emission_probability[s[0]][observations[0]]
        for t in range(0,len(observations)-1):
            score *= transition_probability[s[t]][s[t+1]] * emission_probability[s[t+1]][observations[t+1]]

        if score > result_score:
            result_score = score
            result = s
        print(s, score)
    
    return (result, result_score)
brute_force(states, observations, start_probability, transition_probability, emission_probability)

[['Healthy', 'Healthy', 'Healthy'], ['Healthy', 'Healthy', 'Fever'], ['Healthy', 'Fever', 'Healthy'], ['Healthy', 'Fever', 'Fever'], ['Fever', 'Healthy', 'Healthy'], ['Fever', 'Healthy', 'Fever'], ['Fever', 'Fever', 'Healthy']]
------------------------------------------------------------------------------------------
['Healthy', 'Healthy', 'Healthy'] 0.005879999999999999
['Healthy', 'Healthy', 'Fever'] 0.015119999999999998
['Healthy', 'Fever', 'Healthy'] 0.0010800000000000002
['Healthy', 'Fever', 'Fever'] 0.00972
['Fever', 'Healthy', 'Healthy'] 0.00044800000000000016
['Fever', 'Healthy', 'Fever'] 0.0011520000000000005
['Fever', 'Fever', 'Healthy'] 0.0002880000000000001


(['Healthy', 'Healthy', 'Fever'], 0.015119999999999998)

But! $O(n^m)$ complexity! 

n - nums of observation

m - len of observation

# Viterbi

$O(m \cdot n^2)$

the original paths of algo is a **complete binary tree**, however, there's some identical nodes which can be merged (states)

path_1_hh = start -> health -> health

path_1_fh = start -> fever -> health

...

Finally, we want an argmax, so max(path_1_hh, path_1_fh)

In [4]:
def argmax(xs):
    m = max(xs)
    index = xs.index(m)
    return index, m

def viterbi_decode(states, observations, start_probability, transition_probability, emission_probability):
    
    paths = []
    scores = []
    
    # init
    for s in states:
        scores.append(start_probability[s] * emission_probability[s][observations[0]])
        paths.append([s])
    
    for t in range(1, len(observations)):
        new_path = []
        new_score = []
        for j in range(len(states)):
            index, score = argmax([scores[i] * transition_probability[paths[i][-1]][states[j]] for i in range(len(states))])
                
            score *= emission_probability[states[j]][observations[t]]
    
            new_path.append(paths[index] + [states[j]])
            new_score.append(score)
      
        paths = new_path
        scores = new_score

    index, _ = argmax(scores)
    return paths[index], scores[index]

    
    
viterbi_decode(states, observations, start_probability, transition_probability, emission_probability)

(['Healthy', 'Healthy', 'Fever'], 0.01512)